An implementation of encoded $\ket{A^2} = \ket{\sqrt{-iY}} = e^{-i \frac{\pi}{4}Y} \ket{+} = \ket{1}$ preparation in the Steane code using stim.

In particular, $A^2 = XH$, which will be used to implement the gate in stim.

Maybe easiest to just do $Y$ eigenstate, $\ket{+i}$?

In [3]:
from stim import Circuit
from noisy_circuits import noisy_steane_encoder,noisy_repetition_encoder,noisy_encoded_y
from decoders import steane_transversal_decoder, repetition_transversal_xdecoder, repetition_transversal_zdecoder

The logical $X$ outcome for the repetition-encoded ancilla block indicates which eigenstate the Hadamard test tells us it projected onto.

The logical $Y$ outcome for the Steane-encoded data block will indicate which eigenstate is observed.

If these two observables do not match, then there is a logical error on the output.

Below is a test of the circuit without errors, which should give us no errors and always result in selected trials.

In [19]:
def noisy_repetition_measurement_circuit(circuit:Circuit,block:list[int],perr:float,flag=False) -> Circuit:

    if flag:
        blockmax = int(max(block))
        block.append(blockmax+1)
    
    block_length = len(block)

    first_half = block[:int(block_length/2)]
    second_half = block[int(block_length/2):]

    if block_length%2:
        circuit.append("CNOT", [second_half[-2], second_half[-1]])
        circuit.append("DEPOLARIZE2", [second_half[-2], second_half[-1]],perr)
        circuit.append("DEPOLARIZE1", first_half+second_half[:-2],perr)

    for ii in range(len(first_half)-1,0,-1):
        circuit.append("CNOT", [first_half[ii-1],first_half[ii]])
        circuit.append("DEPOLARIZE2", [first_half[ii-1],first_half[ii]],perr)

        circuit.append("CNOT", [second_half[ii-1],second_half[ii]])
        circuit.append("DEPOLARIZE2", [second_half[ii-1],second_half[ii]],perr)
        
        for jj in range(len(second_half)):
            if not jj in [ii,ii-1]:
                circuit.append("DEPOLARIZE1",[second_half[jj]],perr)
                if jj < len(first_half):
                    circuit.append("DEPOLARIZE1",[first_half[jj]],perr)
                    

    circuit.append("CNOT", [first_half[0],second_half[0]])
    circuit.append("DEPOLARIZE2", [first_half[0],second_half[0]],perr)

    circuit.append("DEPOLARIZE1",first_half[1:]+second_half[1:],perr)

    circuit.append("DEPOLARIZE1", block, perr)
    
    circuit.append("MRX", block[0])
    circuit.append("MRZ", block[1:])

    # if flag:
    #     circuit.append("CNOT",[first_half[-1],second_half[-1]])
    #     circuit.append("DEPOLARIZE2", [first_half[-1],second_half[-1]],perr)
    #     circuit.append("DEPOLARIZE1", first_half[:-1]+second_half[:-1],perr)
        
    #     circuit.append("DEPOLARIZE1",second_half[-1],perr)
    #     circuit.append("MRZ", second_half[-1])

    return circuit


In [18]:
c = Circuit()
c = noisy_repetition_encoder(c,[0,1,2,3,4,5,6],0.0,False)
c = noisy_repetition_measurement_circuit(c,[0,1,2,3,4,5,6],0.0,False)
c.diagram()

q0: -DEPOLARIZE1(0)----------------@-DEPOLARIZE2(0)----------------@-DEPOLARIZE2(0)---------------------------------DEPOLARIZE1(0)---DEPOLARIZE1(0)---DEPOLARIZE1(0)------------------DEPOLARIZE1(0)-@-DEPOLARIZE2(0)----------------@-DEPOLARIZE2(0)----------------
                                   | |                             | |                                                                                                                               | |                             | |
q1: -R--------------DEPOLARIZE1(0)-|-|--------------DEPOLARIZE1(0)-X-DEPOLARIZE2(0)----------------@-DEPOLARIZE2(0)------------------DEPOLARIZE1(0)---DEPOLARIZE1(0)-@-DEPOLARIZE2(0)----------------X-DEPOLARIZE2(0)----------------|-|--------------DEPOLARIZE1(0)-
                                   | |                                                             | |                                                               | |                                                             | |
q2: -R--------------DEPOLARIZE1(0)-|-|--------------DEPOLARIZE1(0)------------------DEPOLARIZE1(0)-X-DEPOLARIZE2(0)------------------DEPOLARIZE1(0)---DEPOLARIZE1(0)-X-DEPOLARIZE2(0)---------------------------------DEPOLARIZE1(0)-|-|--------------DEPOLARIZE1(0)-
                                   | |                                                                                                                                                                                               | |
q3: -R--------------DEPOLARIZE1(0)-X-DEPOLARIZE2(0)------------------@--------------DEPOLARIZE2(0)------------------DEPOLARIZE1(0)---DEPOLARIZE1(0)---DEPOLARIZE1(0)------------------DEPOLARIZE1(0)---@--------------DEPOLARIZE2(0)-X-DEPOLARIZE2(0)----------------
                                                                     |              |                                                                                                                  |              |
q4: -R--------------DEPOLARIZE1(0)------------------DEPOLARIZE1(0)---X--------------DEPOLARIZE2(0)---@--------------DEPOLARIZE2(0)---DEPOLARIZE1(0)---DEPOLARIZE1(0)---@--------------DEPOLARIZE2(0)---X--------------DEPOLARIZE2(0)------------------DEPOLARIZE1(0)-
                                                                                                     |              |                                                  |              |
q5: -R--------------DEPOLARIZE1(0)------------------DEPOLARIZE1(0)------------------DEPOLARIZE1(0)---X--------------DEPOLARIZE2(0)-@-DEPOLARIZE2(0)-@-DEPOLARIZE2(0)---X--------------DEPOLARIZE2(0)------------------DEPOLARIZE1(0)------------------DEPOLARIZE1(0)-
                                                                                                                                   | |              | |
q6: -R--------------DEPOLARIZE1(0)------------------DEPOLARIZE1(0)------------------DEPOLARIZE1(0)------------------DEPOLARIZE1(0)-X-DEPOLARIZE2(0)-X-DEPOLARIZE2(0)------------------DEPOLARIZE1(0)------------------DEPOLARIZE1(0)------------------DEPOLARIZE1(0)-

In [20]:
def decode_sample_output(rec:list[bool]) -> tuple[bool,bool]:
    r"""
    A function that decodes a repetition code (xs) and steane code (yx) observables.
   
    """

    xs = rec[1:8]
    ys = rec[8:]

    mx = sum(xs)%2

    my = steane_transversal_decoder(ys)

    return mx,my

In [ ]:
def decode_repetition_output(rec:list[bool]) -> tuple[bool,bool]:
    r"""
    A function that decodes a repetition code (xs) and steane code (yx) observables.
   
    """

    mx = rec[0]
    zsyndrome = rec[1:]

    # Need to figure out the ZZ correlations measured by the circuit
    # and construct the corresponding Z-check matrix

    # Want to predict and return the X-error that is predicted
    # to have occurred.

    return mx,zsyndrome[0]

In [20]:
error_rates = [ii*10**(-5) for ii in range(15)]

results = []

shots = 10_000_000

for perr in error_rates:
    selected = 0
    errors = 0

    noisy_circ = Circuit()
    
    noisy_circ = construct_noisy_circuit(perr)
    sampler = noisy_circ.compile_sampler()
    sample = sampler.sample(shots)

    for rec in sample:
        if not rec[0]: # post-select on False flag qubit
            mx,my = decode_sample_output(rec)

            if not mx: # initialized with S_DAG
                selected +=1
                errors += not(mx==my)
        
    print(str(perr) + ', ' + str([selected/shots,errors/selected]))
    print()

0.0, [1.0, 0.0]

1e-05, [0.9993373, 5.143408536837362e-05]

2e-05, [0.998694, 9.452344762259511e-05]

3.0000000000000004e-05, [0.9980541, 0.00015169518365787987]

4e-05, [0.9973679, 0.00020303440686230225]

5e-05, [0.9967038, 0.0002448069326112733]

6.000000000000001e-05, [0.9960612, 0.00029556416814549146]

7.000000000000001e-05, [0.9954122, 0.00033744814459778574]

8e-05, [0.994744, 0.00040201298022405767]

9e-05, [0.9941046, 0.00045900602411456503]

0.0001, [0.9934018, 0.0005011064002501304]

0.00011, [0.992803, 0.0005360580094943307]

0.00012000000000000002, [0.9921671, 0.0005866955273965444]

0.00013000000000000002, [0.9914939, 0.0006484154869737474]

0.00014000000000000001, [0.9907923, 0.0007042848435539921]



In [6]:
error_rates = [0,
              1.0*10**(-5), 2.5*10**(-5),5.0*10**(-5),7.5*10**(-5),
              1.0*10**(-4), 2.5*10**(-4),5.0*10**(-4),7.5*10**(-4),
              1.0*10**(-3), 2.5*10**(-3),5.0*10**(-3),7.5*10**(-3),
              1.0*10**(-2)]

results = []

shots = 10_000_000

for perr in error_rates:
    selected = 0
    errors = 0

    noisy_circ = Circuit()
    
    noisy_circ = construct_noisy_circuit(perr)
    sampler = noisy_circ.compile_sampler()
    sample = sampler.sample(shots)

    for rec in sample:
        if not rec[0]: # post-select on False flag qubit
            mx,my = decode_sample_output(rec)

            if not mx: # initialized with S_DAG
                selected +=1
                errors += not(mx==my)
        
    print(str(perr) + ', ' + str([selected/shots,errors/selected]))
    print()

0, [1.0, 0.0]

1e-05, [0.9993503, 4.352828032372632e-05]

2.5e-05, [0.9983614, 0.00010737594622548509]

5e-05, [0.9967139, 0.00022845071188432308]

7.500000000000001e-05, [0.9950673, 0.00034490129461595213]

0.0001, [0.9934573, 0.0004474273831396679]

0.00025, [0.9837322, 0.0011638330025183683]

0.0005, [0.9678256, 0.002347633705907345]

0.00075, [0.9523612, 0.00357700418706684]

0.001, [0.9373937, 0.004841082247512438]

0.0025, [0.8544172, 0.01316780607881021]

0.005, [0.7418412, 0.029681824088497646]

0.0075, [0.6537855, 0.04922027178638865]

0.01, [0.5848589, 0.0710043054829122]

